# **BERT Text Classification for Media Bias Detection**


In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!huggingface-cli login

In [ ]:
# Changed for testing each train/validation split
VAL_SPLIT = 0.1

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import string
regular_punct = list(string.punctuation)

def remove_punctuation(text, punct_list):
  for punc in punct_list:
    if punc in text:
      text = text.replace(punc, ' ')
    return text.strip()

dataset = load_dataset("csv", data_files="dataset_train_val.csv")

texts = dataset["train"]["text"]
for text in texts :
  text = remove_punctuation(text, regular_punct)

labels = dataset["train"]["label"]

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=VAL_SPLIT)

In [ ]:
print(train_labels[0])
print(train_texts[0])

In [ ]:
dataset = load_dataset("csv", data_files="dataset_test.csv")

test_texts = dataset["train"]["text"]
for text in test_texts :
  text = remove_punctuation(text, regular_punct)

text_labels = dataset["train"]["label"]

In [ ]:
print("Train Dataset Size:", len(train_texts))
print("Validation Dataset Size:", len(val_texts))
print("Test Dataset Size:", len(test_texts))

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3
)

# Tuned value for each train/val split
model.config.dropout = 0.5

print(model)

In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-6,
    num_train_epochs=15,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = Dataset(test_encodings, text_labels)

In [ ]:
print(trainer.evaluate(train_dataset))
print(trainer.evaluate(val_dataset))
print(trainer.evaluate(test_dataset))

In [ ]:
!pip install sigopt

In [ ]:
def sigopt_hp_space(trial):
    return [
        {
            "bounds": {"min": 1e-6, "max": 1e-4},
            "name": "learning_rate",
            "type": "double"
        },
        {
            "categorical_values": ["1", "2", "4"],
            "name": "per_device_train_batch_size",
            "type": "categorical"
        },
        {
            "bounds": {"min": 0.01, "max": 0.1},
            "name": "weight_decay",
            "type": "double"
        }
    ]

In [ ]:
def model_init(trial):
  return AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3
  )

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig

trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init
)

In [ ]:
import os
from sigopt import Connection
# Need to export SigOpt account client token here to environment variable "SIGOPT_API_TOKEN" before creating and running project
conn = Connection(client_token="INSERT_API_TOKEN")
os.environ['SIGOPT_API_TOKEN'] = "INSERT_API_TOKEN"
!sigopt create project --project 'huggingface'

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="sigopt",
    hp_space=sigopt_hp_space,
    n_trials=5,
)